In [2]:
import pandas as pd
import numpy as np

df_normal_state = pd.read_csv("data/csvs/flow_data_clean_v2.csv").dropna()
df_abnormal_state = pd.read_csv("data/csvs/flow_data_nmap.csv").dropna()
df_test = pd.read_csv("data/csvs/flow_data_clean.csv").dropna()

df_normal_state.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 77 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             9999 non-null   int64  
 1   Flow Duration                9999 non-null   int64  
 2   Total Fwd Packets            9999 non-null   int64  
 3   Total Backward Packets       9999 non-null   int64  
 4   Total Length of Fwd Packets  9999 non-null   int64  
 5   Total Length of Bwd Packets  9999 non-null   int64  
 6   Fwd Packet Length Max        9999 non-null   int64  
 7   Fwd Packet Length Min        9999 non-null   int64  
 8   Fwd Packet Length Mean       9999 non-null   float64
 9   Fwd Packet Length Std        9999 non-null   float64
 10  Bwd Packet Length Max        9999 non-null   int64  
 11  Bwd Packet Length Min        9999 non-null   int64  
 12  Bwd Packet Length Mean       9999 non-null   float64
 13  Bwd Packet Length 

In [3]:
unique_destination_ports = df_normal_state["Destination Port"].nunique()
print(unique_destination_ports)

4


In [4]:
portas_unicas = df_normal_state["Destination Port"].unique()
print(portas_unicas)

[    0 48780 48794 48796]


In [5]:
casos_diferente_zero = df_normal_state[df_normal_state["Destination Port"] != 0]
quantidade = casos_diferente_zero.shape[0]
print(quantidade)


9


In [6]:
df_normal_state = df_normal_state[df_normal_state["Destination Port"] == 0]


In [7]:
unique_destination_ports = df_normal_state["Destination Port"].nunique()
print(unique_destination_ports)

1


In [12]:
# Concatenar os DataFrames
df_combined = pd.concat([df_normal_state, df_abnormal_state], ignore_index=True)

# Adicionar a coluna Label 
df_combined['Label'] = df_combined['Destination Port'].apply(lambda x: 'BENIGN' if x == 0 else 'ANOMALY')

# Exibir o resultado
df_combined.tail()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
10348,0,2,2,0,164,0,82,82,82.0,0.0,...,82,0,0,0,0,0,0,0,0,BENIGN
10349,0,4,3,0,246,0,82,82,82.0,0.0,...,82,0,0,0,0,0,0,0,0,BENIGN
10350,0,4,3,0,246,0,82,82,82.0,0.0,...,82,0,0,0,0,0,0,0,0,BENIGN
10351,0,0,1,0,82,0,82,82,82.0,0.0,...,82,0,0,0,0,0,0,0,0,BENIGN
10352,0,0,1,0,82,0,82,82,82.0,0.0,...,82,0,0,0,0,0,0,0,0,BENIGN


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Supondo que df_combined já está disponível e contém a coluna 'Label'
# Converter label para valores numéricos
label_map = {'BENIGN': 0, 'ANOMALY': 1}
df_combined['Label_num'] = df_combined['Label'].map(label_map)

# Separar features e target
X = df_combined.drop(columns=['Label', 'Label_num'])
y = df_combined['Label_num']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar pipeline com scaler e classificador RandomForest
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Avaliar o modelo (opcional)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['BENIGN', 'ANOMALY']))

# Salvar o modelo treinado
joblib.dump(pipeline, 'random_forest_anomalia.pkl')


              precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00      2014
     ANOMALY       1.00      1.00      1.00        57

    accuracy                           1.00      2071
   macro avg       1.00      1.00      1.00      2071
weighted avg       1.00      1.00      1.00      2071



['random_forest_anomalia.pkl']